In [1]:
# pretrain MLP using the new dataloader

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from typing import List
import h5py
from torch.utils.data import Dataset, DataLoader
import numpy as np
import preprocessing
from tqdm import tqdm
import keras_core as keras
from keras_core import layers
from keras_core import ops
import matplotlib.pyplot as plt

from data_utils import preprocess_data, H5Dataset

Using TensorFlow backend


2023-10-18 13:17:39.916330: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 13:17:40.980744: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/global/home/users/mfong/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
train_dir = "/global/ml4hep/spss/mfong/transfer_learning/delphes_train/"
train_dir_preprocess = "/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/"
train_filepaths = [train_dir + x for x in os.listdir(train_dir)]
# preprocess_data(train_filepaths, train_dir_preprocess, force=False)
train_preprocess_file_names = os.listdir(train_dir_preprocess)
train_preprocess_filepaths = [train_dir_preprocess + name for name in train_preprocess_file_names]

In [3]:
train_dataset = H5Dataset(train_preprocess_filepaths[:4], transform=None)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=False)
len(train_dataset)

20000000

In [4]:
test_dataset = H5Dataset(train_preprocess_filepaths[5:6], transform=None)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)
len(test_dataset)

5000000

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [6]:
model = Sequential()
model.add(Dense(64, input_shape=(200*7,), activation='relu'))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', "AUC"])

2023-10-18 13:17:48.277214: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 33892 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:41:00.0, compute capability: 8.6


In [7]:
history = model.fit(train_dataloader, batch_size=256, validation_data=test_dataloader, epochs=1)

ValueError: Failed to find data adapter that can handle input: <class 'torch.utils.data.dataloader.DataLoader'>, <class 'NoneType'>

In [11]:
# model = keras.Sequential([
#   keras.Input(shape=(200, 7)),
#   layers.Flatten(),
#   layers.Dense(64, activation='relu'),
#   layers.Dense(8, activation='relu'),
#   layers.Dense(1, activation='sigmoid'),
# ])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "AUC"])

In [12]:
# model.fit(train_dataloader, batch_size=256, validation_data=test_dataloader, epochs=1)
# model.evaluate(val_dataloader)

2023-10-18 12:54:12.598772: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb96c001dd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-18 12:54:12.598833: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2023-10-18 12:54:12.631852: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-18 12:54:14.344489: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:425] Loaded runtime CuDNN library: 8.5.0 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2023-10-18 12:54:14.348252: W tenso

FailedPreconditionError: Graph execution error:

Detected at node 'StatefulPartitionedCall' defined at (most recent call last):
    File "/global/home/users/mfong/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/global/home/users/mfong/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/global/home/users/mfong/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/global/home/users/mfong/anaconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/global/home/users/mfong/anaconda3/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/global/home/users/mfong/anaconda3/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_